In [1]:
import os
import time
import datetime
import joblib
import pandas as pd
import ee
import geemap
from geemap import ml
from sklearn import ensemble

In [2]:
# Initialize Earth Engine
try:
    # ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
    print('Google Earth Engine initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine failed to initialize!', e)
    raise

Google Earth Engine initialized successfully!


In [44]:
INPUT_FOLDER = "./rfr_joblib"
OUTPUT_FOLDER = "./rfr_csv"
AEZ = 16
date = "2025-06-25" # datetime.datetime.today().date()
PRED_VARIABLE = "N"
MODEL_FILE_NAME = f"rfr_model_{date}_AEZ_{AEZ}_{PRED_VARIABLE}.joblib"
rf = joblib.load(os.path.join(INPUT_FOLDER, MODEL_FILE_NAME))
CSV_FILE_NAME = f"rf_trees_t{rf.get_params()['n_estimators']}_d{rf.get_params()['max_depth']}_{date}_AEZ_{AEZ}_{PRED_VARIABLE}.csv"
feature_names = rf.feature_names_in_.tolist()

print(feature_names)

['slope', 'temp', 'RI', 'elevation', 'precipitation', 'sand05', 'silt05', 'sand515', 'longitude', 'latitude']


In [45]:
start_time = time.perf_counter()

# convert the estimator into a list of strings
trees = ml.rf_to_strings(rf, feature_names, processes=8, output_mode="REGRESSION")

end_time = time.perf_counter()
elapsed_time = end_time - start_time
a = datetime.timedelta(seconds=elapsed_time)
print("Time taken : " + str(a))

Time taken : 0:00:52.300453


In [46]:
sizes = [len(trees[i]) for i in range(len(trees))]
print(sizes)

[135673, 133761, 130621, 143236, 135067, 135772, 139145, 135801, 146449, 137483, 138367, 139553, 139633, 133061, 135431, 135193, 137749, 143539, 133764, 141264, 136543, 144257, 140252, 136550, 131540, 136824, 137247, 143553, 133730, 132155]


In [47]:
file_path = os.path.join(OUTPUT_FOLDER, CSV_FILE_NAME)
ml.trees_to_csv(trees, file_path)

In [13]:
df = pd.read_csv(file_path, header=None)

In [14]:
n = len(df)
print(f"Number of trees: {n}")

sizes = [len(df.loc[i].values[0]) for i in range(n)]
print(f"Tree sizes : {sizes}")

Number of trees: 50
Tree sizes : [206954, 194190, 200676, 206700, 205759, 205152, 203323, 197200, 210660, 214174, 209046, 209023, 210324, 211262, 211243, 203550, 203746, 209526, 203799, 199994, 208119, 211635, 212838, 210264, 212206, 208473, 201781, 210976, 204348, 199704, 206705, 210819, 212590, 209304, 206883, 207653, 210636, 198790, 216589, 211387, 208133, 206754, 203134, 199412, 198675, 212385, 202053, 208096, 218634, 198868]
